# Day16_1: MLP (다층 퍼셉트론)

## 학습 목표

**Part 1: 기초**
1. 퍼셉트론의 구조와 동작 원리 이해하기
2. 활성화 함수(Sigmoid, Tanh, ReLU) 비교하기
3. 손실 함수(MSE, Cross-Entropy) 이해하기
4. 경사 하강법 개념 이해하기
5. XOR 문제와 MLP의 필요성 이해하기

**Part 2: 심화**
1. 역전파 알고리즘 개념적으로 이해하기
2. PyTorch로 MLP 구현하기
3. 옵티마이저(SGD, Adam) 비교하기
4. MNIST 손글씨 분류 실습하기

---

## 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| 퍼셉트론 | 신경망의 기본 단위 | 모든 딥러닝 모델의 구성 요소 |
| 활성화 함수 | 비선형성 추가 | ReLU로 깊은 네트워크 학습 가능 |
| 손실 함수 | 학습 목표 정의 | 분류/회귀에 맞는 손실 선택 |
| MLP | 정형 데이터 분류/회귀 | 고객 이탈 예측, 가격 예측 |

**분석가 관점**: MLP는 딥러닝의 기본 구조입니다. CNN, RNN, Transformer 모두 MLP를 기반으로 확장됩니다. 이 개념을 확실히 이해하면 모든 딥러닝 아키텍처의 동작 원리를 파악할 수 있습니다!

---

# Part 1: 기초

---

## 1.1 퍼셉트론 (Perceptron)

### 생물학적 뉴런 비유

```
생물학적 뉴런:           인공 뉴런 (퍼셉트론):
  수상돌기 (입력)    ->    입력 (x1, x2, ...)
  세포체 (처리)      ->    가중합 + 편향
  축삭돌기 (출력)    ->    활성화 함수 -> 출력
```

### 퍼셉트론 수식

```
y = f(w1*x1 + w2*x2 + ... + wn*xn + b)
y = f(sum(wi*xi) + b)
y = f(W @ X + b)

- X: 입력 벡터
- W: 가중치 벡터 (학습되는 파라미터)
- b: 편향 (학습되는 파라미터)
- f: 활성화 함수
```

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

print(f"PyTorch 버전: {torch.__version__}")

In [ ]:
# 단일 퍼셉트론 구현 (NumPy)
def perceptron(x, w, b, activation='step'):
    """
    단일 퍼셉트론
    x: 입력 벡터
    w: 가중치 벡터
    b: 편향
    activation: 활성화 함수 종류
    """
    # 가중합 계산
    z = np.dot(w, x) + b
    
    # 활성화 함수 적용
    if activation == 'step':  # 계단 함수 (원래 퍼셉트론)
        return 1 if z >= 0 else 0
    elif activation == 'sigmoid':
        return 1 / (1 + np.exp(-z))
    else:
        return z  # 선형 (활성화 없음)

# AND 게이트 구현
print("AND 게이트 (퍼셉트론)")
print("="*30)

# 가중치와 편향 (수동 설정)
w_and = np.array([0.5, 0.5])
b_and = -0.7

# 테스트
inputs = [[0, 0], [0, 1], [1, 0], [1, 1]]
for x in inputs:
    output = perceptron(np.array(x), w_and, b_and, 'step')
    print(f"AND({x[0]}, {x[1]}) = {output}")

In [ ]:
# OR 게이트 구현
print("OR 게이트 (퍼셉트론)")
print("="*30)

w_or = np.array([0.5, 0.5])
b_or = -0.2

for x in inputs:
    output = perceptron(np.array(x), w_or, b_or, 'step')
    print(f"OR({x[0]}, {x[1]}) = {output}")

### 실무 예시: 퍼셉트론의 결정 경계

퍼셉트론은 **선형 결정 경계**를 학습합니다:
- w1*x1 + w2*x2 + b = 0
- 이 직선이 두 클래스를 분리

In [ ]:
# AND 게이트의 결정 경계 시각화
x1_range = np.linspace(-0.5, 1.5, 100)

# 결정 경계: w1*x1 + w2*x2 + b = 0
# x2 = -(w1*x1 + b) / w2
x2_boundary_and = -(w_and[0] * x1_range + b_and) / w_and[1]
x2_boundary_or = -(w_or[0] * x1_range + b_or) / w_or[1]

# 데이터 포인트
points = pd.DataFrame({
    'x1': [0, 0, 1, 1],
    'x2': [0, 1, 0, 1],
    'AND': [0, 0, 0, 1],
    'OR': [0, 1, 1, 1]
})

# 시각화
fig = make_subplots(rows=1, cols=2, subplot_titles=['AND 게이트', 'OR 게이트'])

# AND 게이트
colors_and = ['blue' if y == 0 else 'red' for y in points['AND']]
fig.add_trace(
    go.Scatter(x=points['x1'], y=points['x2'], mode='markers', 
               marker=dict(size=15, color=colors_and), name='AND 데이터'),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=x1_range, y=x2_boundary_and, mode='lines',
               line=dict(color='green', dash='dash'), name='결정 경계'),
    row=1, col=1
)

# OR 게이트
colors_or = ['blue' if y == 0 else 'red' for y in points['OR']]
fig.add_trace(
    go.Scatter(x=points['x1'], y=points['x2'], mode='markers',
               marker=dict(size=15, color=colors_or), name='OR 데이터'),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(x=x1_range, y=x2_boundary_or, mode='lines',
               line=dict(color='green', dash='dash'), name='결정 경계'),
    row=1, col=2
)

fig.update_xaxes(range=[-0.5, 1.5], title_text='x1')
fig.update_yaxes(range=[-0.5, 1.5], title_text='x2')
fig.update_layout(title='퍼셉트론의 선형 결정 경계', height=400, showlegend=False)
fig.show()

---

## 1.2 활성화 함수 (Activation Functions)

### 왜 활성화 함수가 필요한가?

활성화 함수 없이 여러 층을 쌓으면:
```
y = W2 @ (W1 @ x) = (W2 @ W1) @ x = W' @ x
```
결국 하나의 선형 변환과 동일! **비선형 활성화 함수**가 있어야 깊은 네트워크의 의미가 있습니다.

In [ ]:
# 활성화 함수 정의
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def tanh(x):
    return np.tanh(x)

def relu(x):
    return np.maximum(0, x)

def leaky_relu(x, alpha=0.01):
    return np.where(x > 0, x, alpha * x)

# 시각화용 데이터
x = np.linspace(-5, 5, 200)

# 활성화 함수 비교 시각화
fig = make_subplots(rows=2, cols=2, subplot_titles=['Sigmoid', 'Tanh', 'ReLU', 'Leaky ReLU'])

fig.add_trace(go.Scatter(x=x, y=sigmoid(x), mode='lines', name='Sigmoid'), row=1, col=1)
fig.add_trace(go.Scatter(x=x, y=tanh(x), mode='lines', name='Tanh'), row=1, col=2)
fig.add_trace(go.Scatter(x=x, y=relu(x), mode='lines', name='ReLU'), row=2, col=1)
fig.add_trace(go.Scatter(x=x, y=leaky_relu(x), mode='lines', name='Leaky ReLU'), row=2, col=2)

fig.update_layout(title='활성화 함수 비교', height=600, showlegend=False)
fig.show()

In [ ]:
# 활성화 함수 특성 비교표
activation_comparison = {
    "활성화 함수": ["Sigmoid", "Tanh", "ReLU", "Leaky ReLU"],
    "출력 범위": ["(0, 1)", "(-1, 1)", "[0, inf)", "(-inf, inf)"],
    "장점": [
        "확률 해석 가능",
        "zero-centered",
        "계산 효율적, 기울기 소실 완화",
        "ReLU의 dying neuron 문제 해결"
    ],
    "단점": [
        "기울기 소실, 느린 학습",
        "기울기 소실 가능",
        "음수 입력에서 뉴런 사망",
        "alpha 하이퍼파라미터 필요"
    ],
    "주 용도": [
        "이진 분류 출력층",
        "RNN 은닉층",
        "대부분의 은닉층 (기본값)",
        "ReLU 대안"
    ]
}

comparison_df = pd.DataFrame(activation_comparison)
print("활성화 함수 비교표")
print("="*80)
print(comparison_df.to_string(index=False))

### Softmax: 다중 클래스 출력

여러 클래스 중 하나를 선택할 때 사용합니다. 모든 출력의 합이 1이 됩니다.

In [7]:
# Softmax 함수
def softmax(x):
    exp_x = np.exp(x - np.max(x))  # 수치 안정성을 위해 max 빼기
    return exp_x / np.sum(exp_x)

# 예시: 3-클래스 분류
logits = np.array([2.0, 1.0, 0.1])
probs = softmax(logits)

print("Softmax 예시 (3-클래스 분류)")
print(f"모델 출력 (logits): {logits}")
print(f"Softmax 후 (확률): {probs}")
print(f"확률 합계: {probs.sum():.4f}")
print(f"예측 클래스: {np.argmax(probs)}")

Softmax 예시 (3-클래스 분류)
모델 출력 (logits): [2.  1.  0.1]
Softmax 후 (확률): [0.65900114 0.24243297 0.09856589]
확률 합계: 1.0000
예측 클래스: 0


---

## 1.3 손실 함수 (Loss Functions)

### 손실 함수의 역할

**손실 함수**: 모델의 예측이 얼마나 틀렸는지 측정
- 작을수록 좋음
- 학습 = 손실 함수 최소화

| 문제 유형 | 손실 함수 | PyTorch |
|----------|----------|----------|
| 회귀 | MSE (Mean Squared Error) | nn.MSELoss() |
| 이진 분류 | BCE (Binary Cross-Entropy) | nn.BCELoss() |
| 다중 분류 | CE (Cross-Entropy) | nn.CrossEntropyLoss() |

In [8]:
# 1. MSE (Mean Squared Error) - 회귀
def mse_loss(y_pred, y_true):
    return np.mean((y_pred - y_true) ** 2)

# 예시
y_true = np.array([3.0, 5.0, 2.5])
y_pred = np.array([2.5, 5.2, 2.0])

print("MSE (회귀)")
print(f"실제값: {y_true}")
print(f"예측값: {y_pred}")
print(f"MSE: {mse_loss(y_pred, y_true):.4f}")

MSE (회귀)
실제값: [3.  5.  2.5]
예측값: [2.5 5.2 2. ]
MSE: 0.1800


In [9]:
# 2. Binary Cross-Entropy - 이진 분류
def bce_loss(y_pred, y_true):
    epsilon = 1e-15  # 수치 안정성
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

# 예시
y_true_bin = np.array([1, 0, 1, 1])
y_pred_prob = np.array([0.9, 0.1, 0.8, 0.7])  # Sigmoid 출력

print("Binary Cross-Entropy (이진 분류)")
print(f"실제 레이블: {y_true_bin}")
print(f"예측 확률: {y_pred_prob}")
print(f"BCE Loss: {bce_loss(y_pred_prob, y_true_bin):.4f}")

Binary Cross-Entropy (이진 분류)
실제 레이블: [1 0 1 1]
예측 확률: [0.9 0.1 0.8 0.7]
BCE Loss: 0.1976


In [10]:
# 3. Cross-Entropy - 다중 분류
def cross_entropy_loss(y_pred_probs, y_true_idx):
    """
    y_pred_probs: Softmax 출력 (확률 분포)
    y_true_idx: 정답 클래스 인덱스
    """
    epsilon = 1e-15
    # 정답 클래스의 확률에 -log 적용
    return -np.log(y_pred_probs[y_true_idx] + epsilon)

# 예시: 3-클래스 분류
y_pred_softmax = np.array([0.7, 0.2, 0.1])  # Softmax 출력
y_true_class = 0  # 정답 클래스

print("Cross-Entropy (다중 분류)")
print(f"예측 확률: {y_pred_softmax}")
print(f"정답 클래스: {y_true_class}")
print(f"CE Loss: {cross_entropy_loss(y_pred_softmax, y_true_class):.4f}")

Cross-Entropy (다중 분류)
예측 확률: [0.7 0.2 0.1]
정답 클래스: 0
CE Loss: 0.3567


### 실무 예시: 손실 함수와 예측 확률의 관계

In [11]:
# 정답 클래스에 대한 예측 확률 vs 손실
probs = np.linspace(0.01, 0.99, 100)
ce_losses = -np.log(probs)

fig = px.line(x=probs, y=ce_losses, labels={'x': '정답 클래스의 예측 확률', 'y': 'Cross-Entropy Loss'})
fig.update_layout(
    title='예측 확률과 Cross-Entropy Loss의 관계',
    template='plotly_white'
)
fig.add_annotation(x=0.9, y=-np.log(0.9), text="좋은 예측\n(낮은 손실)", showarrow=True, arrowhead=1)
fig.add_annotation(x=0.1, y=-np.log(0.1), text="나쁜 예측\n(높은 손실)", showarrow=True, arrowhead=1)
fig.show()

In [12]:
# 경사 하강법 시각화: y = x^2의 최솟값 찾기
def loss_function(x):
    return x ** 2

def gradient(x):
    return 2 * x  # d(x^2)/dx = 2x

# 경사 하강법 실행
x_init = 4.0  # 시작점
lr = 0.1      # 학습률
iterations = 20

x_history = [x_init]
loss_history = [loss_function(x_init)]

x = x_init
for i in range(iterations):
    grad = gradient(x)
    x = x - lr * grad  # 경사 하강
    x_history.append(x)
    loss_history.append(loss_function(x))

print(f"시작점: x = {x_init}")
print(f"최종점: x = {x_history[-1]:.6f}")
print(f"최솟값: f(x) = {loss_history[-1]:.6f}")

시작점: x = 4.0
최종점: x = 0.046117
최솟값: f(x) = 0.002127


In [13]:
# 경사 하강법 경로 시각화
x_range = np.linspace(-5, 5, 100)
y_range = loss_function(x_range)

fig = go.Figure()

# 손실 함수 곡선
fig.add_trace(go.Scatter(x=x_range, y=y_range, mode='lines', name='L(x) = x^2'))

# 경사 하강 경로
fig.add_trace(go.Scatter(
    x=x_history, y=loss_history, mode='markers+lines',
    marker=dict(size=10, color='red'),
    name='경사 하강 경로'
))

fig.update_layout(
    title='경사 하강법: 손실 함수의 최솟값 찾기',
    xaxis_title='파라미터 x',
    yaxis_title='손실 L(x)',
    template='plotly_white'
)
fig.show()

### 학습률(Learning Rate)의 중요성

In [14]:
# 다양한 학습률 비교
learning_rates = [0.01, 0.1, 0.5, 0.99]
colors = ['blue', 'green', 'orange', 'red']

fig = go.Figure()
fig.add_trace(go.Scatter(x=x_range, y=y_range, mode='lines', name='L(x) = x^2', line=dict(color='gray')))

for lr, color in zip(learning_rates, colors):
    x_hist = [4.0]
    x = 4.0
    for _ in range(20):
        x = x - lr * 2 * x
        x_hist.append(x)
    
    y_hist = [h**2 for h in x_hist]
    fig.add_trace(go.Scatter(
        x=x_hist, y=y_hist, mode='markers+lines',
        name=f'lr={lr}', marker=dict(size=6),
        line=dict(color=color)
    ))

fig.update_layout(
    title='학습률(Learning Rate)에 따른 수렴 비교',
    xaxis_title='파라미터 x',
    yaxis_title='손실 L(x)',
    template='plotly_white'
)
fig.show()

print("학습률 가이드:")
print("- 너무 작으면: 수렴이 매우 느림")
print("- 적당하면: 안정적으로 최솟값에 수렴")
print("- 너무 크면: 발산하거나 진동")

학습률 가이드:
- 너무 작으면: 수렴이 매우 느림
- 적당하면: 안정적으로 최솟값에 수렴
- 너무 크면: 발산하거나 진동


---

## 1.5 XOR 문제와 MLP의 필요성

### 단일 퍼셉트론의 한계

XOR 게이트는 **선형으로 분리 불가능**(linearly non-separable)합니다.

```
XOR 진리표:
  (0, 0) -> 0
  (0, 1) -> 1
  (1, 0) -> 1
  (1, 1) -> 0
```

In [15]:
# XOR 데이터
xor_data = pd.DataFrame({
    'x1': [0, 0, 1, 1],
    'x2': [0, 1, 0, 1],
    'XOR': [0, 1, 1, 0]
})

# 시각화
fig = px.scatter(
    xor_data, x='x1', y='x2', color='XOR',
    title='XOR 문제: 선형 결정 경계로 분리 불가능',
    color_continuous_scale='RdBu'
)
fig.update_traces(marker=dict(size=20))
fig.update_layout(template='plotly_white')
fig.add_annotation(text="어떤 직선도 이 데이터를 분리할 수 없습니다!", 
                   x=0.5, y=-0.3, showarrow=False)
fig.show()

In [16]:
# 단일 퍼셉트론으로 XOR 시도 -> 실패
print("단일 퍼셉트론으로 XOR 시도")
print("="*40)

# 어떤 가중치를 써도 XOR를 학습할 수 없음
# w1, w2, b 조합을 시도해봐도 4개 케이스 모두 맞출 수 없음

w_attempts = [
    ([1, 1], -0.5),   # OR처럼
    ([1, 1], -1.5),   # AND처럼
    ([-1, -1], 0.5),  # NOR처럼
]

for w, b in w_attempts:
    print(f"\nw={w}, b={b}")
    correct = 0
    for i, row in xor_data.iterrows():
        x = np.array([row['x1'], row['x2']])
        pred = perceptron(x, np.array(w), b, 'step')
        actual = row['XOR']
        match = "O" if pred == actual else "X"
        print(f"  ({row['x1']}, {row['x2']}) -> 예측: {pred}, 실제: {actual} [{match}]")
        if pred == actual:
            correct += 1
    print(f"  정확도: {correct}/4")

단일 퍼셉트론으로 XOR 시도

w=[1, 1], b=-0.5
  (0, 0) -> 예측: 0, 실제: 0 [O]
  (0, 1) -> 예측: 1, 실제: 1 [O]
  (1, 0) -> 예측: 1, 실제: 1 [O]
  (1, 1) -> 예측: 1, 실제: 0 [X]
  정확도: 3/4

w=[1, 1], b=-1.5
  (0, 0) -> 예측: 0, 실제: 0 [O]
  (0, 1) -> 예측: 0, 실제: 1 [X]
  (1, 0) -> 예측: 0, 실제: 1 [X]
  (1, 1) -> 예측: 1, 실제: 0 [X]
  정확도: 1/4

w=[-1, -1], b=0.5
  (0, 0) -> 예측: 1, 실제: 0 [X]
  (0, 1) -> 예측: 0, 실제: 1 [X]
  (1, 0) -> 예측: 0, 실제: 1 [X]
  (1, 1) -> 예측: 0, 실제: 0 [O]
  정확도: 1/4


### MLP로 XOR 해결

**은닉층(Hidden Layer)** 을 추가하면 비선형 결정 경계를 학습할 수 있습니다.

```
단일 퍼셉트론:       MLP (은닉층 1개):
  x1 ─┬─ y            x1 ─┬─ h1 ─┬─ y
      │                   │      │
  x2 ─┘              x2 ─┴─ h2 ─┘
```

In [17]:
# PyTorch로 XOR 문제 해결
import torch.nn as nn
import torch.optim as optim

# XOR 데이터 준비
X_xor = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float32)
y_xor = torch.tensor([[0], [1], [1], [0]], dtype=torch.float32)

# MLP 모델 정의
class XOR_MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(2, 4),    # 입력 2 -> 은닉 4
            nn.ReLU(),
            nn.Linear(4, 1),    # 은닉 4 -> 출력 1
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.model(x)

# 모델, 손실 함수, 옵티마이저
model_xor = XOR_MLP()
criterion = nn.BCELoss()
optimizer = optim.Adam(model_xor.parameters(), lr=0.1)

print("XOR MLP 모델 구조:")
print(model_xor)

XOR MLP 모델 구조:
XOR_MLP(
  (model): Sequential(
    (0): Linear(in_features=2, out_features=4, bias=True)
    (1): ReLU()
    (2): Linear(in_features=4, out_features=1, bias=True)
    (3): Sigmoid()
  )
)


In [18]:
# 학습
epochs = 1000
losses = []

for epoch in range(epochs):
    # 순전파
    y_pred = model_xor(X_xor)
    loss = criterion(y_pred, y_xor)
    
    # 역전파
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    
    if epoch % 200 == 0:
        print(f"Epoch {epoch:4d}: Loss = {loss.item():.4f}")

# 최종 예측
print("\n===== XOR 학습 완료 =====")
with torch.no_grad():
    predictions = model_xor(X_xor)
    for i in range(4):
        x = X_xor[i].numpy()
        pred = predictions[i].item()
        actual = y_xor[i].item()
        print(f"XOR({int(x[0])}, {int(x[1])}) = {pred:.3f} (실제: {int(actual)})")

Epoch    0: Loss = 0.7233
Epoch  200: Loss = 0.0019
Epoch  400: Loss = 0.0007
Epoch  600: Loss = 0.0004
Epoch  800: Loss = 0.0002

===== XOR 학습 완료 =====
XOR(0, 0) = 0.000 (실제: 0)
XOR(0, 1) = 1.000 (실제: 1)
XOR(1, 0) = 1.000 (실제: 1)
XOR(1, 1) = 0.000 (실제: 0)


---

# Part 2: 심화

---

## 2.1 역전파 알고리즘 (Backpropagation)

### 개념적 이해

**역전파**: 손실 함수의 그래디언트를 출력층에서 입력층 방향으로 계산하는 알고리즘

```
순전파 (Forward):
  x -> h1 -> h2 -> ... -> y_pred -> Loss

역전파 (Backward):
  dL/dx <- dL/dh1 <- dL/dh2 <- ... <- dL/dy_pred <- dL/dL = 1
```

### 체인 룰 (Chain Rule)

복합 함수의 미분: 각 함수의 미분을 곱합니다.

```
y = f(g(x))
dy/dx = dy/dg * dg/dx = f'(g(x)) * g'(x)
```

In [19]:
# 체인 룰 예시: y = (2x + 1)^2
# g(x) = 2x + 1
# f(g) = g^2
# dy/dx = dy/dg * dg/dx = 2g * 2 = 4(2x + 1)

x_val = torch.tensor([3.0], requires_grad=True)

# 순전파
g = 2 * x_val + 1  # g(x) = 2x + 1
y = g ** 2         # f(g) = g^2

# 역전파
y.backward()

print("체인 룰 검증: y = (2x + 1)^2")
print(f"x = {x_val.item()}")
print(f"g = 2x + 1 = {g.item()}")
print(f"y = g^2 = {y.item()}")
print(f"\nAutograd로 계산한 dy/dx: {x_val.grad.item()}")
print(f"수동 계산: 4(2x + 1) = 4 * {g.item()} = {4 * g.item()}")

체인 룰 검증: y = (2x + 1)^2
x = 3.0
g = 2x + 1 = 7.0
y = g^2 = 49.0

Autograd로 계산한 dy/dx: 28.0
수동 계산: 4(2x + 1) = 4 * 7.0 = 28.0


### 직관적 설명: 오차가 어디서 왔는가?

역전파는 "최종 오차가 각 파라미터에 얼마나 기인했는가?"를 계산합니다.

- 기여도가 큰 파라미터 -> 많이 수정
- 기여도가 작은 파라미터 -> 조금 수정

In [20]:
# 간단한 2층 네트워크에서 역전파 흐름 확인
torch.manual_seed(42)

# 단순 네트워크: 입력(1) -> 은닉(2) -> 출력(1)
simple_net = nn.Sequential(
    nn.Linear(1, 2),
    nn.ReLU(),
    nn.Linear(2, 1)
)

# 데이터
x = torch.tensor([[1.0]])
y_true = torch.tensor([[3.0]])

# 순전파
y_pred = simple_net(x)
loss = nn.MSELoss()(y_pred, y_true)

print("순전파:")
print(f"  입력: {x.item()}")
print(f"  예측: {y_pred.item():.4f}")
print(f"  실제: {y_true.item()}")
print(f"  손실: {loss.item():.4f}")

# 역전파
loss.backward()

print("\n역전파 (각 레이어의 그래디언트):")
for name, param in simple_net.named_parameters():
    print(f"  {name}: grad shape = {param.grad.shape}")
    print(f"         grad = {param.grad.numpy()}")

순전파:
  입력: 1.0
  예측: -0.1769
  실제: 3.0
  손실: 10.0927

역전파 (각 레이어의 그래디언트):
  0.weight: grad shape = torch.Size([2, 1])
         grad = [[ 0.9843937 ]
 [-0.90660995]]
  0.bias: grad shape = torch.Size([2])
         grad = [ 0.9843937  -0.90660995]
  2.weight: grad shape = torch.Size([1, 2])
         grad = [[ -3.369211 -11.110398]]
  2.bias: grad shape = torch.Size([1])
         grad = [-6.353812]


---

## 2.2 PyTorch로 MLP 구현하기

### MLP 아키텍처 설계

```
입력층 (784) -> 은닉층1 (128) -> 은닉층2 (64) -> 출력층 (10)
         ↓           ↓            ↓           ↓
      [ReLU]      [ReLU]       [ReLU]     [Softmax]
```

In [21]:
# MLP 클래스 정의
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim, dropout=0.2):
        """
        MLP (Multi-Layer Perceptron)
        
        Args:
            input_dim: 입력 차원
            hidden_dims: 은닉층 차원 리스트 [128, 64]
            output_dim: 출력 차원
            dropout: 드롭아웃 비율
        """
        super().__init__()
        
        layers = []
        prev_dim = input_dim
        
        # 은닉층 추가
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            prev_dim = hidden_dim
        
        # 출력층
        layers.append(nn.Linear(prev_dim, output_dim))
        
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.model(x)

# MNIST용 MLP 생성
mlp = MLP(
    input_dim=784,        # 28x28 이미지
    hidden_dims=[128, 64],  # 은닉층
    output_dim=10,        # 0~9 숫자
    dropout=0.2
)

print("MNIST 분류용 MLP:")
print(mlp)

MNIST 분류용 MLP:
MLP(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=64, out_features=10, bias=True)
  )
)


In [22]:
# 파라미터 수 계산
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"총 파라미터 수: {count_parameters(mlp):,}")

# 각 레이어별 파라미터
print("\n레이어별 파라미터:")
for name, param in mlp.named_parameters():
    print(f"  {name}: {param.numel():,} ({param.shape})")

총 파라미터 수: 109,386

레이어별 파라미터:
  model.0.weight: 100,352 (torch.Size([128, 784]))
  model.0.bias: 128 (torch.Size([128]))
  model.3.weight: 8,192 (torch.Size([64, 128]))
  model.3.bias: 64 (torch.Size([64]))
  model.6.weight: 640 (torch.Size([10, 64]))
  model.6.bias: 10 (torch.Size([10]))


---

## 2.3 옵티마이저 비교 (SGD vs Adam)

### 주요 옵티마이저

| 옵티마이저 | 특징 | 장점 | 단점 |
|-----------|------|-----|------|
| SGD | 기본 경사 하강법 | 단순, 일반화 좋음 | 느린 수렴, lr 민감 |
| SGD+Momentum | 관성 추가 | 진동 감소, 빠른 수렴 | 하이퍼파라미터 추가 |
| Adam | 적응적 학습률 | 빠른 수렴, lr 덜 민감 | 일반화 약할 수 있음 |

In [ ]:
optim

In [23]:
# 간단한 회귀 문제로 옵티마이저 비교
torch.manual_seed(42)

# 데이터 생성
X = torch.linspace(-3, 3, 100).reshape(-1, 1)
y = torch.sin(X) + 0.1 * torch.randn_like(X)

def train_with_optimizer(optimizer_name, lr=0.01, epochs=500):
    """다양한 옵티마이저로 학습"""
    torch.manual_seed(42)
    
    # 간단한 MLP
    model = nn.Sequential(
        nn.Linear(1, 32),
        nn.ReLU(),
        nn.Linear(32, 16),
        nn.ReLU(),
        nn.Linear(16, 1)
    )
    
    # 옵티마이저 선택
    if optimizer_name == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=lr)
    elif optimizer_name == 'SGD+Momentum':
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    elif optimizer_name == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=lr)
    
    criterion = nn.MSELoss()
    losses = []
    
    for epoch in range(epochs):
        y_pred = model(X)
        loss = criterion(y_pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item())
    
    return losses, model

# 각 옵티마이저로 학습
sgd_losses, sgd_model = train_with_optimizer('SGD', lr=0.1)
momentum_losses, momentum_model = train_with_optimizer('SGD+Momentum', lr=0.1)
adam_losses, adam_model = train_with_optimizer('Adam', lr=0.01)

print("최종 손실 비교:")
print(f"  SGD:          {sgd_losses[-1]:.4f}")
print(f"  SGD+Momentum: {momentum_losses[-1]:.4f}")
print(f"  Adam:         {adam_losses[-1]:.4f}")

최종 손실 비교:
  SGD:          0.0234
  SGD+Momentum: 0.0076
  Adam:         0.0070


In [24]:
# 학습 곡선 비교
fig = go.Figure()

fig.add_trace(go.Scatter(y=sgd_losses, mode='lines', name='SGD'))
fig.add_trace(go.Scatter(y=momentum_losses, mode='lines', name='SGD+Momentum'))
fig.add_trace(go.Scatter(y=adam_losses, mode='lines', name='Adam'))

fig.update_layout(
    title='옵티마이저별 학습 곡선 비교',
    xaxis_title='Epoch',
    yaxis_title='Loss (MSE)',
    template='plotly_white',
    yaxis_type='log'
)
fig.show()

---

## 2.4 MNIST 손글씨 분류 실습

### 데이터셋 소개

- **MNIST**: 손글씨 숫자(0~9) 이미지 데이터셋
- 훈련 데이터: 60,000개
- 테스트 데이터: 10,000개
- 이미지 크기: 28x28 픽셀 (흑백)

In [25]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# 데이터 변환 정의
transform = transforms.Compose([
    transforms.ToTensor(),  # PIL Image -> Tensor, 0~1 정규화
    transforms.Normalize((0.1307,), (0.3081,))  # MNIST 평균, 표준편차
])

# MNIST 데이터셋 로드
train_dataset = torchvision.datasets.MNIST(
    root='./data', train=True, download=True, transform=transform
)
test_dataset = torchvision.datasets.MNIST(
    root='./data', train=False, download=True, transform=transform
)

print(f"훈련 데이터: {len(train_dataset)}개")
print(f"테스트 데이터: {len(test_dataset)}개")
print(f"이미지 shape: {train_dataset[0][0].shape}")
print(f"클래스 수: 10 (0~9)")

훈련 데이터: 60000개
테스트 데이터: 10000개
이미지 shape: torch.Size([1, 28, 28])
클래스 수: 10 (0~9)


In [26]:
# 샘플 이미지 시각화
fig = make_subplots(rows=2, cols=5, subplot_titles=[f"Label: {train_dataset[i][1]}" for i in range(10)])

for i in range(10):
    img, label = train_dataset[i]
    img_np = img.squeeze().numpy()
    
    row = i // 5 + 1
    col = i % 5 + 1
    
    fig.add_trace(
        go.Heatmap(z=img_np, colorscale='gray', showscale=False),
        row=row, col=col
    )

fig.update_layout(title='MNIST 샘플 이미지', height=400)
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False, autorange='reversed')
fig.show()

In [28]:
# DataLoader 생성
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"배치 크기: {batch_size}")
print(f"훈련 배치 수: {len(train_loader)}")
print(f"테스트 배치 수: {len(test_loader)}")

배치 크기: 64
훈련 배치 수: 938
테스트 배치 수: 157


In [29]:
# MNIST 분류 MLP 모델
class MNIST_MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.model = nn.Sequential(
            nn.Linear(784, 128),   # 28*28 = 784
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 10)      # 10개 클래스
        )
    
    def forward(self, x):
        x = self.flatten(x)  # (batch, 1, 28, 28) -> (batch, 784)
        return self.model(x)

# 모델 생성
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MNIST_MLP().to(device)

# 손실 함수, 옵티마이저
criterion = nn.CrossEntropyLoss()  # Softmax + NLL Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

print(f"Device: {device}")
print(f"파라미터 수: {count_parameters(model):,}")
print(model)

Device: cpu
파라미터 수: 109,386
MNIST_MLP(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=64, out_features=10, bias=True)
  )
)


In [30]:
# 학습 함수
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for X_batch, y_batch in loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        # 순전파
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        
        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # 통계
        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += y_batch.size(0)
        correct += predicted.eq(y_batch).sum().item()
    
    return total_loss / len(loader), 100. * correct / total

# 평가 함수
def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            
            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += y_batch.size(0)
            correct += predicted.eq(y_batch).sum().item()
    
    return total_loss / len(loader), 100. * correct / total

In [31]:
# 학습 실행
epochs = 10

train_losses, train_accs = [], []
test_losses, test_accs = [], []

print("MNIST MLP 학습 시작")
print("="*50)

for epoch in range(epochs):
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)
    
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    test_losses.append(test_loss)
    test_accs.append(test_acc)
    
    print(f"Epoch {epoch+1:2d}/{epochs}: "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | "
          f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")

MNIST MLP 학습 시작
Epoch  1/10: Train Loss: 0.3449, Train Acc: 89.64% | Test Loss: 0.1305, Test Acc: 95.97%
Epoch  2/10: Train Loss: 0.1676, Train Acc: 94.99% | Test Loss: 0.0998, Test Acc: 96.88%
Epoch  3/10: Train Loss: 0.1281, Train Acc: 96.13% | Test Loss: 0.0942, Test Acc: 97.18%
Epoch  4/10: Train Loss: 0.1112, Train Acc: 96.62% | Test Loss: 0.0890, Test Acc: 97.16%
Epoch  5/10: Train Loss: 0.0986, Train Acc: 96.95% | Test Loss: 0.0938, Test Acc: 97.23%
Epoch  6/10: Train Loss: 0.0919, Train Acc: 97.19% | Test Loss: 0.0774, Test Acc: 97.77%
Epoch  7/10: Train Loss: 0.0812, Train Acc: 97.46% | Test Loss: 0.0795, Test Acc: 97.80%
Epoch  8/10: Train Loss: 0.0771, Train Acc: 97.61% | Test Loss: 0.0881, Test Acc: 97.45%
Epoch  9/10: Train Loss: 0.0743, Train Acc: 97.70% | Test Loss: 0.0722, Test Acc: 97.84%
Epoch 10/10: Train Loss: 0.0723, Train Acc: 97.69% | Test Loss: 0.0739, Test Acc: 97.64%


In [32]:
# 학습 곡선 시각화
fig = make_subplots(rows=1, cols=2, subplot_titles=['손실 (Loss)', '정확도 (Accuracy)'])

# 손실
fig.add_trace(go.Scatter(y=train_losses, mode='lines+markers', name='Train Loss'), row=1, col=1)
fig.add_trace(go.Scatter(y=test_losses, mode='lines+markers', name='Test Loss'), row=1, col=1)

# 정확도
fig.add_trace(go.Scatter(y=train_accs, mode='lines+markers', name='Train Acc'), row=1, col=2)
fig.add_trace(go.Scatter(y=test_accs, mode='lines+markers', name='Test Acc'), row=1, col=2)

fig.update_xaxes(title_text='Epoch')
fig.update_yaxes(title_text='Loss', row=1, col=1)
fig.update_yaxes(title_text='Accuracy (%)', row=1, col=2)
fig.update_layout(title='MNIST MLP 학습 곡선', height=400, template='plotly_white')
fig.show()

In [ ]:
# 예측 결과 시각화
model.eval()
with torch.no_grad():
    # 테스트 데이터에서 샘플 추출
    sample_images, sample_labels = next(iter(test_loader))
    sample_images = sample_images[:10].to(device)
    sample_labels = sample_labels[:10]
    
    outputs = model(sample_images)
    _, predictions = outputs.max(1)

# 시각화
fig = make_subplots(rows=2, cols=5)

for i in range(10):
    img = sample_images[i].cpu().squeeze().numpy()
    pred = predictions[i].item()
    actual = sample_labels[i].item()
    color = 'green' if pred == actual else 'red'
    
    row = i // 5 + 1
    col = i % 5 + 1
    
    fig.add_trace(
        go.Heatmap(z=img, colorscale='gray', showscale=False),
        row=row, col=col
    )
    fig.add_annotation(
        text=f"Pred: {pred} (Actual: {actual})",
        x=0.5+i*28*28*10, y=-0.15+i*2ß8*28*10,
        showarrow=False, font=dict(color=color)
    )

fig.update_layout(title='MNIST 예측 결과', height=400)
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False, autorange='reversed')
fig.show()

---

## 실습 퀴즈

**난이도**: (쉬움) ~ (어려움)

---

### Q1. 퍼셉트론 출력 계산하기 

**문제**: 아래 퍼셉트론의 출력을 계산하세요 (Sigmoid 활성화 사용).

- 입력: x = [1.0, 2.0]
- 가중치: w = [0.5, -0.5]
- 편향: b = 0.1

**기대 결과**: Sigmoid 출력 값

In [ ]:
import numpy as np

x = np.array([1.0, 2.0])
w = np.array([0.5, -0.5])
b = 0.1

# 여기에 코드를 작성하세요


### Q2. 활성화 함수 선택하기 

**문제**: 다음 상황에 적합한 활성화 함수를 선택하고, 이유를 설명하세요.

1. 은닉층 (일반적인 경우)
2. 이진 분류의 출력층 (0~1 확률)
3. 다중 분류의 출력층 (여러 클래스 중 하나 선택)

In [ ]:
# 여기에 답과 이유를 작성하세요


### Q3. 손실 함수 계산하기 

**문제**: 다음 예측과 실제 값으로 BCE Loss를 수동 계산하세요.

- 예측 확률: y_pred = [0.9, 0.3, 0.8]
- 실제 레이블: y_true = [1, 0, 1]

In [ ]:
import numpy as np

y_pred = np.array([0.9, 0.3, 0.8])
y_true = np.array([1, 0, 1])

# 여기에 코드를 작성하세요


### Q4. 경사 하강법 구현하기 

**문제**: 함수 f(x) = (x - 3)^2의 최솟값을 경사 하강법으로 찾으세요.

- 시작점: x = 10
- 학습률: 0.1
- 반복 횟수: 50

In [ ]:
# 여기에 코드를 작성하세요


### Q5. XOR MLP 수정하기 

**문제**: XOR 문제를 해결하는 MLP를 은닉층 2개로 수정하세요.

- 구조: 입력(2) -> 은닉(8) -> 은닉(4) -> 출력(1)
- 활성화: ReLU (은닉), Sigmoid (출력)

In [ ]:
import torch
import torch.nn as nn

# 여기에 코드를 작성하세요


### Q6. 역전파 그래디언트 확인하기 

**문제**: y = 2x^3 - 3x^2 + x 함수에서 x=2일 때 dy/dx를 Autograd로 계산하고, 수동 계산과 비교하세요.

힌트: dy/dx = 6x^2 - 6x + 1

In [ ]:
import torch

# 여기에 코드를 작성하세요


### Q7. 옵티마이저 선택하기 

**문제**: 아래 상황에 적합한 옵티마이저를 선택하고 이유를 설명하세요.

1. 빠른 프로토타이핑이 필요한 경우
2. 일반화 성능이 중요한 최종 모델
3. 학습률 튜닝 시간이 부족한 경우

In [ ]:
# 여기에 답과 이유를 작성하세요


### Q8. MLP 구조 설계하기 

**문제**: Fashion MNIST (28x28 이미지, 10개 클래스) 분류를 위한 MLP를 설계하세요.

요구사항:
- 은닉층 3개 (256, 128, 64)
- 드롭아웃 0.3
- BatchNorm 적용

In [ ]:
import torch
import torch.nn as nn

# 여기에 코드를 작성하세요


### Q9. MNIST 정확도 향상하기 

**문제**: MNIST MLP 모델을 수정하여 테스트 정확도 98% 이상을 달성하세요.

힌트:
- 은닉층 크기 증가
- 에포크 증가
- 학습률 스케줄러 사용
- 배치 정규화 추가

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# 여기에 코드를 작성하세요


### Q10. 종합: 이진 분류 MLP 파이프라인 

**문제**: sklearn의 breast cancer 데이터셋으로 이진 분류 MLP를 구축하세요.

요구사항:
1. 데이터 표준화 (StandardScaler)
2. MLP 모델 (은닉층 2개)
3. Adam 옵티마이저, BCE Loss
4. 학습 곡선 시각화
5. 테스트 정확도 출력

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim

# 데이터 로드
data = load_breast_cancer()
X, y = data.data, data.target

# 여기에 코드를 작성하세요


---

## 학습 정리

### Part 1: 기초 핵심 요약

| 개념 | 핵심 내용 | 실무 활용 |
|-----|----------|----------|
| 퍼셉트론 | y = f(W @ X + b) | 신경망의 기본 단위 |
| 활성화 함수 | 비선형성 추가 | ReLU (은닉), Sigmoid/Softmax (출력) |
| 손실 함수 | 예측 오차 측정 | MSE (회귀), CE (분류) |
| 경사 하강법 | w = w - lr * gradient | 손실 최소화 |
| XOR 문제 | 단일 퍼셉트론의 한계 | MLP로 해결 |

### Part 2: 심화 핵심 요약

| 개념 | 핵심 내용 | 언제 사용? |
|-----|----------|----------|
| 역전파 | 체인 룰로 그래디언트 계산 | 모든 신경망 학습 |
| MLP | 입력 -> 은닉층들 -> 출력 | 정형 데이터 분류/회귀 |
| SGD vs Adam | 기본 vs 적응적 학습률 | Adam이 기본, SGD는 일반화 |
| MNIST | 손글씨 분류 벤치마크 | 딥러닝 입문 |

### MLP 학습 핵심 패턴

```python
model = MLP(input_dim, hidden_dims, output_dim)
criterion = nn.CrossEntropyLoss()  # 분류
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(epochs):
    for X_batch, y_batch in dataloader:
        outputs = model(X_batch)          # 순전파
        loss = criterion(outputs, y_batch) # 손실 계산
        
        optimizer.zero_grad()   # 그래디언트 초기화
        loss.backward()         # 역전파
        optimizer.step()        # 파라미터 업데이트
```

### 실무 팁

1. **활성화 함수**: 은닉층에는 ReLU, 출력층은 문제에 맞게 (Sigmoid, Softmax, 없음)
2. **학습률**: Adam은 0.001, SGD는 0.01~0.1로 시작
3. **과적합 방지**: Dropout, 조기 종료, 데이터 증강
4. **배치 크기**: 32~128이 일반적, 메모리와 속도 트레이드오프
5. **디버깅**: 먼저 작은 데이터로 과적합 확인 (모델이 학습 가능한지)